In [1]:
# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pickle
import numpy as np
np.set_printoptions(suppress=True)

import sys
sys.path.append("../")
import bayesflow as beef
from experiment import ModelComparisonExperiment

/Users/lukas/opt/miniconda3/envs/beef/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
experiment = ModelComparisonExperiment()

INFO:root:Trainer initialization: No generative model provided. Only offline learning mode is available!
INFO:root:Initialized empty loss history.
INFO:root:Initialized networks from scratch.


# Training

In [4]:
# read simulated data for offline training
with open('../data/sim_data_offline_learning.pkl', 'rb') as f:
    sim_data = pickle.load(f)

In [5]:
history = experiment.run(simulations_dict=sim_data)

Training epoch 1:   0%|          | 0/3125 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Validation

# Evaluation